## Import 2015-2019 Data

In [13]:
import pandas as pd
import openpyxl
import os
import glob


In [14]:

# Replace 'your_file.xlsx' with the correct path to your Excel file
file_path = r'Tabular_Sem1-A15129\Plot_Level\2015-2019 Wheat Main Plot Data (All traits)_anonymized.xlsx'

df_crop = pd.read_excel(file_path, engine="openpyxl")
df_crop['SowingDate'] = pd.to_datetime(df_crop['SowingDate'])
df_crop['HarvestDate'] = pd.to_datetime(df_crop['HarvestDate'])
print(df_crop.head())


   Year   Name State MET Analysis Mega Region RegionName  TrialCode  \
0  2015  Wheat   NSW                 Northern        N/E  trial_719   
1  2015  Wheat   NSW                 Northern        N/E  trial_719   
2  2015  Wheat   NSW                 Northern        N/E  trial_719   
3  2015  Wheat   NSW                 Northern        N/E  trial_719   
4  2015  Wheat   NSW                 Northern        N/E  trial_719   

  SiteDescription SowingDate HarvestDate Abandoned  ...  \
0         Bellata 2015-05-19  2015-11-24     False  ...   
1         Bellata 2015-05-19  2015-11-24     False  ...   
2         Bellata 2015-05-19  2015-11-24     False  ...   
3         Bellata 2015-05-19  2015-11-24     False  ...   
4         Bellata 2015-05-19  2015-11-24     False  ...   

   Septoria Tritici score  Shattering %  Standability  Tillering Score  \
0                     NaN           NaN           NaN              NaN   
1                     NaN           NaN           NaN              NaN

## Import weather data

In [81]:
import os
import glob
import pandas as pd

# Set the folder path
folder_path = r'Tabular_Sem1-A15129\Weather_data\SILO_Point_Data'

# Print the absolute folder path to check if it's correct
print(f"Absolute Folder Path: {os.path.abspath(folder_path)}")

# Use glob to find all .csv files (including in subdirectories)
file_paths = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)

# Print the file paths to verify that files are detected
print(f"Detected Files: {file_paths}")

# Create an empty dictionary to store the data
df_trial_weather_dict = {}
print(file_paths[0])
# Load the data into a DataFrame
# df = pd.read_csv(file_paths[0], skiprows=30, sep='\s+', header=1)
# df = df.iloc[1:]
# df.head()
#Loop through each file and load it into a pandas DataFrame
for file_path in file_paths:
    # Extract the file name without the path
    file_name = os.path.basename(file_path)
    
    # Remove the '_SILOweather' part from the file name to use as the key
    key = file_name.replace('_SILOweather.csv', '')
    
    try:
        # Load the data into a DataFrame
        df = pd.read_csv(file_path, skiprows=30, sep='\s+', header=1)
        df = df.iloc[1:]
        # Store the DataFrame in the dictionary with the modified file name as the key
        df_trial_weather_dict[key] = df

    except pd.errors.EmptyDataError:
        # Handle the case where the file is empty
        print(f"Warning: The file '{file_name}' is empty and was skipped.")
    except Exception as e:
        # Handle other potential errors
        print(f"Error reading file '{file_name}': {e}")

# Print the first few rows of one of the DataFrames to confirm it's working
if df_trial_weather_dict:
    print(df_trial_weather_dict)
else:
    print("No dataframes were loaded.")


<>:34: SyntaxWarning: invalid escape sequence '\s'
<>:34: SyntaxWarning: invalid escape sequence '\s'
C:\Users\harry\AppData\Local\Temp\ipykernel_34144\1975512855.py:34: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path, skiprows=30, sep='\s+', header=1)


Absolute Folder Path: c:\Users\harry\Documents\Harry\UQ\DATA7901\Tabular_Sem1-A15129\Weather_data\SILO_Point_Data
Detected Files: ['Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1000_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1001_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1002_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1003_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1004_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1005_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1006_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1007_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1008_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Data\\trial_1009_SILOweather.csv', 'Tabular_Sem1-A15129\\Weather_data\\SILO_Point_Da

In [82]:
for trial_code, df in df_trial_weather_dict.items():
    print(df.columns)
    #df['Date'] = pd.to_datetime(df['Date'])

aggregated_results = {}

for _, row in df_crop.iterrows():
    trial_code = row['TrialCode']
    
    # Check if trial exists in the timescale dictionary
    if trial_code in df_trial_weather_dict:
        df_timescale = df_trial_weather_dict[trial_code]
        
        # Merge the trial timescale data with the crop data
        df_merged = pd.merge(df_timescale, row.to_frame().T, on='TrialCode', how='inner')
        
        # Filter rows based on the sowing and harvest dates
        df_filtered = df_merged[(df_merged['Date'] >= df_merged['SowingDate']) & (df_merged['Date'] <= df_merged['HarvestDate'])]
        
        # Perform aggregation (e.g., sum of Value for each TrialCode)
        aggregated_data = df_filtered.groupby('TrialCode').agg({'Value': 'sum'}).reset_index()
        
        # Store the result in the dictionary
        aggregated_results[trial_code] = aggregated_data

for trial_code, result in aggregated_results.items():
    print(f"Aggregated data for {trial_code}:\n{result}\n")

Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2'],
      dtype='object')
Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2'],
      dtype='object')
Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2'],
      dtype='object')
Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2'],
      dtype='object')
Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2'],
      dtype='object')
Index(['Date', 'Day', 'T.Max', 'Smx', 'T.Min', 'Smn', 'Rain', 'Srn', 'Evap',
       'Sev', 'Radn', 'Ssl', 'VP', 'Svp', 'RHmaxT', 'RHminT', 'Date2

KeyError: 'TrialCode'

## Select useful features

In [9]:
columns_to_keep = [
    'Year', 'Name', 'State', 'RegionName', 'SiteDescription', 'TrialCode', 
    'SowingDate', 'HarvestDate', 'Abandoned', 
    'Range', 'Row', 'CultivarID', 
    'Harvest Length', 'Harvest Width', 'Kg/Plot', 'GrainYield_gm2', 'Residual'
]

# Filter the DataFrame to keep only the selected columns
df_filtered = df[columns_to_keep]
df_filtered.head()

,Year,Name,State,RegionName,SiteDescription,TrialCode,SowingDate,HarvestDate,Abandoned,Range,Row,CultivarID,Harvest Length,Harvest Width,Kg/Plot,GrainYield_gm2,Residual
0,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19 00:00:00,2015-11-24 00:00:00,False,1,1,ea9c07b5,8.4,2.1,7.46,422.902494,0.060914
1,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19 00:00:00,2015-11-24 00:00:00,False,1,2,cd19a3a0,8.4,2.1,6.04,342.403628,-0.164896
2,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19 00:00:00,2015-11-24 00:00:00,False,1,3,e7cf3b4d,8.3,2.1,7.12,408.491107,0.001737
3,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19 00:00:00,2015-11-24 00:00:00,False,1,4,41230145,8.4,2.1,6.92,392.290249,0.096828
4,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19 00:00:00,2015-11-24 00:00:00,False,1,5,201d5469,8.5,2.1,6.24,349.579832,-0.097151
